# Анализ A/B теста для интернет-магазина

Для анализа предоставлены данные с действиями пользователей в интернет-магазине за период с 7 декабря 2020 по 4 января 2021 года для групп А и В.

Цель исследования: проверить соответствие данных с техническим заданием, провести исследовательский анализ и оценить результаты A/B-теста по данным интернет-магазина.

**Описание данных:**

Таблица matketing_events (календарь маркетинговых событий на 2020 год):
- name — название маркетингового события;
- regions — регионы, в которых будет проводиться рекламная кампания;
- start_dt — дата начала кампании;
- finish_dt — дата завершения кампании.

Таблица users (все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года):
- user_id — идентификатор пользователя;
- first_date — дата регистрации;
- region — регион пользователя;
- device — устройство, с которого происходила регистрация.

Таблица events (все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года):
- user_id — идентификатор пользователя;
- event_dt — дата и время события;
- event_name — тип события;
- details — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.

Таблица participants (список участников тестов):
- user_id — идентификатор пользователя;
- ab_test — название теста;
- group — группа пользователя.

**Техническое задание**
* Название теста: recommender_system_test;
* Группы: А (контрольная), B (новая платёжная воронка);
* Дата запуска: 2020-12-07;
* Дата остановки набора новых пользователей: 2020-12-21;
* Дата остановки: 2021-01-04;
* Аудитория: 15% новых пользователей из региона EU;
* Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
* Ожидаемое количество участников теста: 6000.
* Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%:
  * конверсии в просмотр карточек товаров — событие product_page
  * просмотры корзины — product_cart
  * покупки — purchase.

## План работы

1. [Изучение данных](#1)

2. [Предобработка данных](#2)

    2.1 [Замена типа данных](#2.1)

    2.2 [Поиск дубликатов](#2.2)

3. [Проверка соответствия с техническим заданием](#3)

    3.1 [Участники теста](#3.1)

    3.2 [Даты регистрации пользователей и проведения теста](#3.2)

    3.3 [Процент аудитории](#3.3)

4. [Исследовательский анализ данных](#4)

    4.1 [Объединение таблиц](#4.1)

    4.2 [Распределение количества событий в выборках на пользователя](#4.2)

    4.3 [Распределение количества событий в выборках по дням](#4.3)

    4.4 [Воронка событий](#4.4)

    4.5 [Учёт особенностей данных](#4.5)

5. [Оценка результатов A/B-тестирования](#5)

6. [Общие выводы](#6)

<a name="1"><h2>1 Изучение данных</h2></a>

Импортируем библиотеки:

In [1]:
import pandas as pd
import numpy as np
import math as mth
from datetime import datetime, timedelta
import datetime as dt
from scipy import stats as st
import plotly.graph_objects as go
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

Прочитаем данные и сохраним их в переменные:

In [2]:
matketing_events = pd.read_csv('/content/final_ab_marketing_events.csv')
users =  pd.read_csv('/content/final_ab_new_users.csv')
events = pd.read_csv('/content/final_ab_events.csv')
participants = pd.read_csv('/content/final_ab_participants.csv')

Выведем первые 10 строк таблицы matketing_events и посмотрим общую информацию:

In [3]:
matketing_events.head(10)

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


In [4]:
matketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes


Таблица содержит 14 строк и 4 столбца, пропусков нет.

Посмотрим таблицу users:

In [5]:
users.head(10)

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone
5,137119F5A9E69421,2020-12-07,N.America,iPhone
6,62F0C741CC42D0CC,2020-12-07,APAC,iPhone
7,8942E64218C9A1ED,2020-12-07,EU,PC
8,499AFACF904BBAE3,2020-12-07,N.America,iPhone
9,FFCEA1179C253104,2020-12-07,EU,Android


In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB


Таблица содержит 61733 строк и 4 столбца, пропусков нет.

Посмотрим таблицу events:

In [7]:
events.head(10)

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99
5,831887FE7F2D6CBA,2020-12-07 06:50:29,purchase,4.99
6,6B2F726BFD5F8220,2020-12-07 11:27:42,purchase,4.99
7,BEB37715AACF53B0,2020-12-07 04:26:15,purchase,4.99
8,B5FA27F582227197,2020-12-07 01:46:37,purchase,4.99
9,A92195E3CFB83DBD,2020-12-07 00:32:07,purchase,4.99


In [8]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB


Таблица содержит 440317 строк и 4 столбца, пропуски есть в поле 'details', подсчитаем их долю:

In [9]:
round(events['details'].isna().mean()*100, 2)

85.75

Так как согласно описанию данных в этом поле может содержиться информация о сумме покупки, то возможно пропуски у пользователей, которые не совершали оплату. Проверим это предположение и сгруппируем данные по событиям:

In [10]:
events.groupby('event_name').count()

,user_id,event_dt,details
event_name,,,
login,189552,189552,0
product_cart,62462,62462,0
product_page,125563,125563,0
purchase,62740,62740,62740


Заполненные данные в колонке 'details' только для события о покупке 'purchase', поэтому предположение о том, что пропуски у клиентов, которые не совершили оплату, верное.

Посмотрим таблицу participants:

In [11]:
participants.head()

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


In [12]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB


В таблице 18268 строк и 3 столбца, пропусков нет.

<div style="border: solid green 2px; padding: 20px">

**Выводы**
* В таблице 'events' 85.75% пропусков в столбце 'details'. Оставляем их без изменения, так как пропуски относятся к пользователям, которые не совершали оплату.
* Столбцы с датой в таблицах 'matketing_events', 'users' и 'events' записаны в формате object, поэтому их следует привести в формат даты.</div>

<a name="2"><h2>2 Предобработка данных</h2></a>

<a name="2.1"><h3>2.1 Замена типа данных</h3></a>

Поменяем формат в таблицах 'matketing_events', 'users' и 'events' на формат даты:

In [13]:
users['first_date'] = pd.to_datetime(users['first_date'])
events['event_dt'] = pd.to_datetime(events['event_dt'], format='%Y-%m-%d %H:%M:%S')
matketing_events['start_dt'] = pd.to_datetime(matketing_events['start_dt'])
matketing_events['finish_dt'] = pd.to_datetime(matketing_events['finish_dt'])

Проверим, что формат изменился:

In [14]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     61733 non-null  object        
 1   first_date  61733 non-null  datetime64[ns]
 2   region      61733 non-null  object        
 3   device      61733 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.9+ MB


In [15]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     440317 non-null  object        
 1   event_dt    440317 non-null  datetime64[ns]
 2   event_name  440317 non-null  object        
 3   details     62740 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 13.4+ MB


In [16]:
matketing_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes


Во всех таблицах формат успешно изменён.

<a name="2.2"><h3>2.2 Поиск дубликатов</h3></a>

Проверим таблицу 'users' на наличие дубликатов:

In [17]:
users.duplicated().sum()

0

In [18]:
users['user_id'].duplicated().sum()

0

Проверим таблицу 'matketing_events':

In [19]:
matketing_events.duplicated().sum()

0

Проверим таблицу 'participants':

In [20]:
participants.duplicated().sum()

0

Дубликатов в таблицах нет.

<div style="border: solid green 2px; padding: 20px">
    
**Выводы**
    
* Столбцы с датами привели в соответствующий формат.
* Проверили таблицы на наличие дубликатов.</div>

<a name="3"><h2>3 Проверка соответствия с техническим заданием</h2></a>

<a name="3.1"><h3>3.1 Участники теста</h3></a>

**Название теста: recommender_system_test**

Посмотрим в каких тестах участвовали пользователи:

In [21]:
participants['ab_test'].value_counts()

interface_eu_test          11567
recommender_system_test     6701
Name: ab_test, dtype: int64

В таблице содержатся данные по участникам двух тестов.

Напишем функцию для поиска пересекающихся пользователей:

In [22]:
def check_intersection(table, data_one, data_two):
    group_one = table.query(data_one)['user_id']
    group_two = table.query(data_two)['user_id']
    return set(group_one).intersection(set(group_two))

Проверим, есть ли пересечения в тестах interface_eu_test и recommender_system_test:

In [23]:
len(check_intersection(participants, 'ab_test == "recommender_system_test"', 'ab_test == "interface_eu_test"'))

1602

Одновременно в обоих тестах участвовали 1602 пользователя. Чтобы оценить, могло ли проведение второго теста оказать влияние на  тест recommender_system_test, посмотрим в какую группу попали такие пользователи, в контрольную или тестовую.

Создадим таблицу, в которой оставим только пересекающихся пользователей:

In [24]:
intersecting_users = check_intersection(participants, 'ab_test == "recommender_system_test"', 'ab_test == "interface_eu_test"')
participants_both_tests = participants.query("user_id in @intersecting_users")

Проверим, сколько участников из теста interface_eu_test попали в контрольную и тестовую группу recommender_system_test:

In [25]:
participants_both_tests.query('ab_test == "recommender_system_test"').groupby(['ab_test', 'group']).count()

user_id
ab_test                 group         
recommender_system_test A          921
                        B          681

В группу A попал 921 пользователь, и если тест interface_eu_test проходил задолго до теста recommender_system_test, то на этих пользователей тест не оказал влияния. Если тесты проходили одновременно, то на контрольную группу теста recommender_system_test могли оказать влияние те участники, которые находились в группе B теста interface_eu_test, проверим их количество:

In [26]:
len(check_intersection(participants_both_tests,
                   'ab_test == "recommender_system_test" and group == "A"',
                   'ab_test == "interface_eu_test" and group == "B"'))

439

Если тесты проходили в одно время, то на поведение 439 пользователей группы A теста 'recommender_system_test' могло оказать влияние участие этих пользователей в тестовой группе 'interface_eu_test'.

В группу B теста recommender_system_test попал 681 человек и конкурирующий тест мог повлиять на активность пользователей, если они тоже находились в тестовой группе, проверим их число: 

In [27]:
len(check_intersection(participants_both_tests,
                   'ab_test == "recommender_system_test" and group == "B"',
                   'ab_test == "interface_eu_test" and group == "B"'))

344

Таких участников 344.

Так как согласно техническому заданию нам нужны только участники теста 'recommender_system_test', то сделаем срез таблицы и оставим только таких пользователей:

In [28]:
participants = participants.query('ab_test == "recommender_system_test"')

Проверим, что в таблице остались участники теста 'recommender_system_test':

In [29]:
participants['ab_test'].value_counts()

recommender_system_test    6701
Name: ab_test, dtype: int64

**Группы: А — контрольная, B — новая платёжная воронка**

Напишем функцию, которая будет подсчитывать количество пользователей по группам:

In [30]:
def users_group(data):
    table = data.groupby('group')['user_id'].agg(['nunique'])
    table['percent'] = (table['nunique']/data['user_id'].nunique())*100
    return table

Проверим равномерность распределения по тестовым группам:

In [31]:
users_group(participants)

,nunique,percent
group,,
A,3824,57.07
B,2877,42.93


Выборки неравномерные, в тестируемой группе примерно на 14% меньше пользователей.

Проверим, есть ли пользователи которые попали и в группу А, и в группу В:

In [32]:
len(check_intersection(participants, 'group == "A"', 'group == "B"'))

0

Пересекающихся пользователей нет.

<a name="3.2"><h3>3.2 Даты регистрации пользователей и проведения теста</h3></a>

**Дата начала регистрации пользователей 2020-12-07 и остановка набора новых пользователей 2020-12-21**

Проверим даты регистрации пользователей:

In [33]:
print('Минимальная дата регистрации: ', users['first_date'].min())
print('Максимальная дата регистрации: ', users['first_date'].max())

Минимальная дата регистрации:  2020-12-07 00:00:00
Максимальная дата регистрации:  2020-12-23 00:00:00


Дата начала регистрации верная, а максимальная выходит за пределы 21 декабря. Так как согласно техническому заданию дата остановки набора новых пользователей 2020-12-21, то уберем из таблицы пользователей, которые были зарегистрированы 22 и 23 числа.

In [34]:
users = users.query('first_date < 20201222')

Проверим, что период набора новых пользователей равен 14 дней:

In [35]:
users['first_date'].max() - users['first_date'].min()

Timedelta('14 days 00:00:00')

**Дата запуска: 2020-12-07, дата остановки: 2021-01-04**

Посмотрим даты совершённых событий:

In [36]:
print('Минимальная дата события: ', events['event_dt'].min())
print('Максимальная дата события: ', events['event_dt'].max())

Минимальная дата события:  2020-12-07 00:00:33
Максимальная дата события:  2020-12-30 23:36:33


Дата запуска совпадает с датой первого события, а последнее событие зарегистрировано на несколько дней раньше конца проведения теста.

Проверим, проводились ли маркетинговые мероприятия в период проведения теста в регионе EU:

In [37]:
matketing_events[(matketing_events['regions'].str.contains("EU"))
                 & (matketing_events['start_dt'] >= '20201207')
                 & (matketing_events['start_dt'] <= '20210104')]

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03


В период теста в регионе EU проводилось одно мероприятие, связанное с праздниками Новый год и Рождество.

<a name="3.3"><h3>3.3 Процент аудитории</h3></a>

**Аудитория: 15% новых пользователей из региона EU**

Подсчитаем количество уникальных пользователей в регионе EU:

In [38]:
users.groupby('region')['user_id'].agg(['count'])

,count
region,
APAC,2883
CIS,2900
EU,42340
N.America,8347


Оставим в таблице только пользователей из региона EU:

In [39]:
users = users.query('region == "EU"')

Посмотрим число участников теста, которое как ожидается в техническом задании равно 6000.

In [40]:
len(participants)

6701

Участников на 701 человека больше.

Рассчитаем долю клиентов, поделив количество участников теста на общее число пользователей из региона EU:

In [41]:
round(len(participants)/len(users)*100, 2)

15.83

Доля аудитории немного выше 15%, возможно в таблице 'participants' есть клиенты из других регионов. Объединим таблицы 'users' и 'participants' и посмотрим:

In [42]:
users_participants = participants.merge(users, on = 'user_id')
users_participants.head()

,user_id,group,ab_test,first_date,region,device
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC
1,A7A3664BD6242119,A,recommender_system_test,2020-12-20,EU,iPhone
2,DABC14FDDFADD29E,A,recommender_system_test,2020-12-08,EU,Mac
3,04988C5DF189632E,A,recommender_system_test,2020-12-14,EU,iPhone
4,482F14783456D21B,B,recommender_system_test,2020-12-14,EU,PC


Подсчитаем число пользователей:

In [43]:
len(users_participants)

6351

Участников стало меньше, рассчитаем долю пользователей еще раз:

In [44]:
len(users_participants)/len(users)*100

15.0

Теперь доля ровно 15%.

Посмотрим, изменилось ли соотношений пользователей по группам:

In [45]:
users_group(users_participants)

,nunique,percent
group,,
A,3634,57.22
B,2717,42.78


Соотношение практически не изменилось.

<div style="border: solid green 2px; padding: 20px">
    
**Выводы**

* В таблице 'participants' оставлены только участники теста 'recommender_system_test'.
* Выборки A и B неравномерные, в группе A на 14% больше пользователей.  
* По датам регистрации убрали из таблицы пользователей, которые были зарегистрированы после 21 декабря.
* После 30 декабря не было зарегистрировано ни одного события, хотя срок завершения теста 4 января.
* В период  проведения теста (25 декабря) проводилось одно маркетинговое мероприятие.
* Аудитория теста - 15% из региона EU, как и указано в техническом задании.</div>

<a name="4"><h2>4 Исследовательский анализ данных</h2></a>

<a name="4.1"><h3>4.1 Объединение таблиц</h3></a>

Для проведения исследовательского анализа объединим данные таблиц 'users_participants' и 'events':

In [46]:
df = users_participants.merge(events, on = 'user_id', how = 'left')
df.head()

,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,purchase,99.99
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-25 00:04:56,purchase,4.99
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:29,product_cart,NaN
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-25 00:04:57,product_cart,NaN
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,product_page,NaN


Посмотрим общую информацию о таблице:

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26290 entries, 0 to 26289
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     26290 non-null  object        
 1   group       26290 non-null  object        
 2   ab_test     26290 non-null  object        
 3   first_date  26290 non-null  datetime64[ns]
 4   region      26290 non-null  object        
 5   device      26290 non-null  object        
 6   event_dt    23420 non-null  datetime64[ns]
 7   event_name  23420 non-null  object        
 8   details     3196 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(6)
memory usage: 2.0+ MB


Подсчитаем процент пропусков в поле с названием событий:

In [48]:
round(df['event_name'].isna().mean()*100, 2)

10.92

Почти в 11% случаев клиенты не совершили никаких действий. Заполним пропуски для подсчета количества таких событий по группам:

In [49]:
df['event_name'] = df['event_name'].fillna('no_info')

Проверим их число и соотношение по группам:

In [50]:
users_group(df.query('event_name == "no_info"'))

,nunique,percent
group,,
A,1030,35.89
B,1840,64.11


В группе B на 28.22% больше случаев, когда пользователи зарегистрировались, но больше не совершили никаких действий. Следует рассмотреть причины, из-за которых пользователи после регистрации не перешли к этапу авторизации, возможно в результате изменений была допущена какая-то техническая ошибка, которая не позволила некоторым пользователям пройти дальше.

Так как для результатов теста важно изменение данных по метрикам, а такие пользователи приравниваются к неучаствовавшим в тесте, то не будем их учитывать при дальнейшем анализе.

Добавим столбец с количеством дней, прошедших со дня регистрации до совершения события:

In [51]:
df['count_days'] = (df['event_dt'] - df['first_date']).dt.days

Проверим, что столбец был добавлен:

In [52]:
df.head()

,user_id,group,ab_test,first_date,region,device,event_dt,event_name,details,count_days
0,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,purchase,99.99,0.00
1,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-25 00:04:56,purchase,4.99,18.00
2,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:29,product_cart,NaN,0.00
3,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-25 00:04:57,product_cart,NaN,18.00
4,D1ABA3E2887B6A73,A,recommender_system_test,2020-12-07,EU,PC,2020-12-07 14:43:27,product_page,NaN,0.00


Уберем из таблицы строки, где количество дней с момента регистрации до совершения действия превышет 14:

In [53]:
df = df.query('count_days <= 14')

Посмотрим параметры таблицы:

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22828 entries, 0 to 26289
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     22828 non-null  object        
 1   group       22828 non-null  object        
 2   ab_test     22828 non-null  object        
 3   first_date  22828 non-null  datetime64[ns]
 4   region      22828 non-null  object        
 5   device      22828 non-null  object        
 6   event_dt    22828 non-null  datetime64[ns]
 7   event_name  22828 non-null  object        
 8   details     3123 non-null   float64       
 9   count_days  22828 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(6)
memory usage: 1.9+ MB


При срезе также были удалены пользователи без событий. Проверим число оставшихся пользователей:

In [55]:
df['user_id'].nunique()

3481

В результате осталось 3481 уникальных пользователя.

Посмотрим соотношение пользователей по группам:

In [56]:
total_in_groups = users_group(df)
total_in_groups

,nunique,percent
group,,
A,2604,74.81
B,877,25.19


Соотношение в группах стало еще более неравномерное, в группе А теперь почти на 50% клиентов больше.

Проверим долю пользователей из региона EU:

In [57]:
round(df['user_id'].nunique()/len(users)*100, 2)

8.22

Стало примерно на 7% меньше пользователей из региона EU.

Подсчитаем процент пропусков в поле 'details' таблицы 'df':

In [58]:
round(df['details'].isna().mean()*100, 2)

86.32

Примерно в 14% случаев действие пользователя завершалось покупкой.

<a name="4.2"><h3>4.2 Распределение количества событий в выборках на пользователя</h3></a>

Создадим сводную таблицу с количеством событий на каждого пользователя:

In [59]:
users_count_events = df.groupby(['group','user_id'])['event_name'].agg(['count']).reset_index()
users_count_events.head()

,group,user_id,count
0,A,0010A1C096941592,12
1,A,00341D8401F0F665,2
2,A,003DF44D7589BBD4,15
3,A,00505E15A9D81546,5
4,A,006E3E4E232CE760,6


Построим boxplot и посмотрим на распределение количества событий по пользователям для групп A и B:

In [60]:
fig = px.box(users_count_events, x="group", y="count", color="group")
fig.update_layout(width=700, height=500,  title = 'Распределение количества событий для групп A и B')
fig.show()

Для группы A межквартильный размах от 4 до 9 событий, медиана равна 6. В группе В пользователи совершают меньше действий, здесь межквартильный размах от 3 до 8 событий, медиана равна 4. Максимальный выброс в обеих группах 24 события.

Подсчитаем сколько действий в среднем совершает пользователь из каждой группы:

In [61]:
print('Среднее число событий пользователя в группе A: ', users_count_events.query('group == "A"')['count'].mean().round(2))
print('Среднее число событий пользователя в группе B: ', users_count_events.query('group == "B"')['count'].mean().round(2))

Среднее число событий пользователя в группе A:  6.9
Среднее число событий пользователя в группе B:  5.53


В среднем пользователи из группы А совершают примерно на одно событие больше.

<a name="4.3"><h3>4.3 Распределение количества событий в выборках по дням</h3></a>

Создадим сводную таблицу с количеством событий в выборках по датам:

In [62]:
group_event_dt = df.groupby(['group','event_dt'])['event_name'].agg(['count']).reset_index()
group_event_dt.head()

,group,event_dt,count
0,A,2020-12-07 00:14:01,2
1,A,2020-12-07 00:16:00,3
2,A,2020-12-07 00:31:08,2
3,A,2020-12-07 00:35:15,2
4,A,2020-12-07 00:39:13,1


Построим гистограмму распределения событий:

In [63]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=group_event_dt.query('group=="A"')["event_dt"], name="Группа A"))
fig.add_trace(go.Histogram(x=group_event_dt.query('group=="B"')["event_dt"], name="Группа B"))
fig.update_layout(width=950, height=600, xaxis_title='Дата', yaxis_title="Количество событий", title="Распределение количества событий в выборках по дням")
fig.show()

В первые дни теста число событий по группам отличалось незначительно, но после 14 декабря пользователи из группы А стали совершать намного больше действий, при этом в группе В подобных всплесков в тот же период не наблюдалось и разница по количеству стала значительной. Наибольшее число для группы A было зафиксировано 21 декабря - 327 событий, а у группы B в то же время 78 событий.

<a name="4.4"><h3>4.4 Воронка событий</h3></a>

Напишем функцию для подсчета количества пользователей, которые совершали каждое из событий по группам и рассчитаем их долю:

In [64]:
def group_event(group_name):
    table = df.query('group == @group_name').groupby(['event_name'])['user_id'].agg(['nunique']).sort_values(by='nunique', ascending=False)
    table['percent'] = (table['nunique']*100/df.query('group == @group_name')['user_id'].nunique()).round(2)
    return table.reset_index()

Создадим таблицу для группы А:

In [65]:
group_event_A = group_event("A")
group_event_A

,event_name,nunique,percent
0,login,2604,100.00
1,product_page,1685,64.71
2,purchase,833,31.99
3,product_cart,782,30.03


Создадим таблицу для группы В:

In [66]:
group_event_B = group_event("B")
group_event_B

,event_name,nunique,percent
0,login,876,99.89
1,product_page,493,56.21
2,purchase,249,28.39
3,product_cart,244,27.82


На каждом шаге видно ухудшение по каждой метрике в группе В по сравнению с группой А.

Последовательность событий:

* login - авторизация
* product_page - просмотр карточек товаров
* product_cart - корзина
* purchase - оплата

Так как в событии product_cart значение меньше, чем в покупках, значит некоторые пользователи оформляли заказ сразу без добавления товара в корзину.

Поменяем местами шаги product_cart и purchase для групп A и B:

In [67]:
group_event_A = group_event_A.reindex([0,1,3,2])
group_event_B = group_event_B.reindex([0,1,3,2])

Построим график воронки событий:

In [68]:
fig = go.Figure()
fig.add_trace(go.Funnel(name = 'A', y=group_event_A['event_name'],
              x=group_event_A['nunique'], textposition = "inside",
              textinfo = "value+percent initial+percent previous",
              texttemplate = "%{value}<br>%{percentInitial:.1%}(percent of initial)<br>%{percentPrevious:.1%}(percent of previous)"))
fig.add_trace(go.Funnel(name = 'B',y=group_event_B['event_name'],
              x=group_event_B['nunique'],
              textinfo = "value+percent initial+percent previous",
              texttemplate = "%{value}<br>%{percentInitial:.1%}(percent of initial)<br>%{percentPrevious:.1%}(percent of previous)"))
fig.update_layout(width = 1000, title="Воронка событий для групп A и B")
fig.show()

Согласно воронке:
* После авторизации (login), при переходе на страницу с продуктами (product_page) в группе B теряется на 8,4% больше пользователей, чем в группе А. 
* До оплаты в тестируемой группе доходит 28,4% пользователей, в группе А на 3,6% больше.

<a name="4.5"><h3>4.5 Учёт особенностей данных</h3></a>

Выделим следующие особенности данных:

* Пользователи изначально неравномерно распределены по выборкам, для группы А было выбрано 57.22% пользователей, для группы В - 42.78%. После удаления пользователей, которые не прошли дальше регистрации, разница в соотношении групп стала еще больше: в группе А - 74.81% и в группе В - 25.19% пользователей.
* Согласно техническому заданию предполагалось участие 6000 новых пользователей из региона EU. В результате осталось 3481 пользователя, и вместо 15% из региона EU осталось 8,22%.
* Дата остановки теста 2021-01-04, но уже после 30 декабря вероятно произошла какая-то ошибка и данные о действиях пользователей перестали записываться, либо были утеряны.
* В период теста проводилось одно маркетинговое мероприятие, которое могло повлиять на результаты.

<div style="border: solid green 2px; padding: 20px">
    
**Выводы**

* В результате объединения таблиц были обнаружили пользователи, которые зарегистрировались, но не совершали никаких действий, таких пропусков около 11%.
* Сделали срез таблицы и оставили только те события, которые были совершены в течение 2 недель с момента регистрации. При этом пользователи без событий тоже были удалены, всего 2519. В итоге в группе A осталось 2604 участника (74.81%) и в группе B - 877 человек (25.19%).
* В среднем пользователь группы A совершает 6.9 событий, а пользователь группы B - 5.53. Медиана в тестируемой группе на 2 события меньше, чем в группе A.
* Распределение количества событий в выборках по дням показало, что клиенты группы A стали значительно активние после 14 декабря.
* По метрикам группа В показывает более плохие результаты по сравнению с группой А, наибольшая разница 8,4% при переходе в каталог товаров (product_page).
* Учли особенности данных, которые могли повлиять на результаты теста.</div>

<a name="5"><h2>5 Оценка результатов A/B-тестирования</h2></a>

По результам A/В тестирования определили, что:

* ожидаемого эффекта нет, за 14 дней с момента регистрации пользователи  не показали улучшение ни по одной метрике, тогда как предполагалось улучшение не менее, чем на 10% в каждой метрике;
* с учетом всех особенностей данных, можно утверждать, что тест проведен некорректно.

Для проверки, существует ли разница в долях между выборками, сформулируем гипотезы:

* H₀: Нет статистически значимых различий в долях между группами A и B
* H₁: Есть статистически значимые различия в долях между группами A и B

Для проверки гипотез объединим таблицы 'group_event_A' и 'group_event_B' для сравнения групп по событиям:

In [69]:
events_by_groups = group_event_A.merge(group_event_B, on='event_name')
events_by_groups.set_index('event_name', inplace=True)
events_by_groups.columns = ['A', 'percent_A', 'B', 'percent_B']
events_by_groups

,A,percent_A,B,percent_B
event_name,,,,
login,2604,100.00,876,99.89
product_page,1685,64.71,493,56.21
product_cart,782,30.03,244,27.82
purchase,833,31.99,249,28.39


Сравним группы по событиям от общего числа пользователей в каждой группе, используя Z-test для проверки гипотез о равенстве долей:

In [70]:
events = ['login', 'product_page', 'product_cart', 'purchase']
for event in events:
        
    # массивы с данными о количестве пользователей в группах и их общим количеством:
    successes = np.array([events_by_groups.loc[event, 'A'],
                   events_by_groups.loc[event, 'B']])
    trials = np.array([total_in_groups.loc['A', 'nunique'],
                      total_in_groups.loc['B', 'nunique']])
        
    # пропорция успехов в первой группе и во второй группе:
    p1 = successes[0]/trials[0]
    p2 = successes[1]/trials[1] 
        
    #пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])
    # разница пропорций в датасетах
    difference = p1 - p2
        
    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(
    p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))
        
    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)
    # критический уровень статистической значимости 0,05
    alpha = .05
    # считаем p_value
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    bonferroni_alpha = alpha / 3
        
    # проверяем гипотезу:
    print('Событие: ', event, ', p-значение:', p_value, sep='') 
    if p_value < bonferroni_alpha:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print(
        'Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными')
    print()

Событие: login, p-значение:0.08481837035887363
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: product_page, p-значение:6.942739359416805e-06
Отвергаем нулевую гипотезу: между долями есть значимая разница

Событие: product_cart, p-значение:0.21469192029582396
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными

Событие: purchase, p-значение:0.04652482738393027
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными



<div style="border: solid green 2px; padding: 20px">

**Вывод**
  .
* Для событий login, product_cart и purchase значение p-value больше 0.05, поэтому нулевую гипотезу не отвергаем.
* Между долями события product_page есть статистически значимая разница, поэтому нулевую гипотезу принять нельзя.</div>

<a name="6"><h2>6 Общие выводы</h2></a>

В результате работы изучили предоставленные данные и проанализировали результаты A/B теста.

На этапе проверки были выявлены несоответствия требований технического задания с полученными данными, а именно: некорректное распределение пользователей, в котором в тестовой группе оказалось на 14% меньше участников, отсутствие записей с действиями пользователей после 30 декабря по 4 января 2021, а также проведение маркетингового мероприятия. При этом было обнаружено большое число пользователей (2604), которые не совершали никаких действий, кроме регистрации.

После того как в работе были оставлены только те события, срок которых  не превышал 14 дней с момента регистрации, количество участников стало 3841 и выборки оказались еще более неравномерные. Так в группе A стало 74.81%, а в группе B 25,19% пользователей. Аудитория из региона EU уменьшилась примерно на 7% и осталось 8,22% вместо 15%.

Данные A/B теста показали следующие результаты:

* По распределению событий в выборках по пользователям участники A теста чаще совершают события. За тестовый период половина пользователей совершила от 4 до 9 действий, медиана в группе равна 6, среднее значение 6,9. В группе B у половины пользователей от 3 до 8 действий, медиана равна 4 и среднее значение 5,53;

* По распределению количества событий по дням группа A тоже заметно более активная. С 7 по 13 января разница была небольшая, но после 14 числа пользователи из группы A стали проводить значительно больше действий, так, например, 14 числа в 6:00 в группе A было совершено 173 события, а в группе B 53. Максимальный всплекс зафиксирован 21 декабря (327 против 78), затем показатели стали постепенно снижаться, но все равно оставались выше группы B;

* По воронке событий группа B также показала более низкие результаты:
  * После авторизации (login) в просмотр карточек товаров (product_page) переходят только 56.21% пользователей, тогда как в группе A - 64.71%;
  * Из карточек товаров (product_page) в корзину (product_cart) переходят на 3,6% меньше пользователей, для группы B доля 28.39%, для группы A - 31.99%;
  * Из корзины (product_cart) к оплате (purchase) переходят еще на 2,21% меньше клиентов, для группы B доля 27.82% и для группы A - 30.03%.

В итоге сделан вывод, что ожидаемого эффекта не было достигнуто, по метрикам улучшений нет.

Но в связи с тем, что данные не соответствовали требованиям технического задания, нельзя считать верными результаты теста.

Рекомендуется провести тестирование повторно и учесть следующее:
* На этапе выбора участников убедиться в равности их соотношений;
* Проверить, что пользователи не принимают участие в других тестах; 
* Проверять корректность сбора данных и сохранения записей;
* Во время проведения теста не проводить маркетинговые мероприятия;
* Не проводить тест в праздничный период.

Также проверили статистическую разницу долей между группами z-критерием, который показал, что есть значимая разница между долями на шаге product_page.